# Wheelchair Accessibility

There are two fields within the GTFS spec that communicate wheelchair accessibility:

- trips.txt - `wheelchair_accessible` - indicates the vehicle servicing the trip is wheelchair accessible.
    - This one is straightforward because all of our vehicles are required to be wheelchair accessible to begin with.
- stops.txt - `wheelchair_boarding` - indicates the stop is wheelchair accessible.
    - This will be easier for rail stations, since they are all built to be wheelchair accessible. The tricky part is figuring out which specific rows to flag as wheelchair accessible in a station with complex hierarchies.
    - Bus stop accessibility will be a challenge.  Even if we had resources to do a big push to survey all the bus stops, we currently have no way of identifying when new data needs to be gathered.  This could potentially be addressed in creative ways - satellite imagery, crowdsourced data, notifications from city partners, etc.

## Setup

Set up by downloading and unzipping the data that will need to be modified.

In [17]:
import pandas as pd
import os

# Rail Stops
os.system("wget -O data/rail/stops.txt https://gitlab.com/LACMTA/gtfs_rail/-/raw/master/stops.txt")
# Rail Trips
os.system("wget -O data/rail/trips.txt https://gitlab.com/LACMTA/gtfs_rail/-/raw/master/trips.txt")

# Download Bus GTFS
os.system("wget -O data/bus/gtfs.zip https://gitlab.com/LACMTA/gtfs_bus/-/raw/weekly-updated-service/gtfs_bus.zip")

# Unzip Bus Stops
os.system("unzip -o data/bus/gtfs.zip stops.txt -d data/bus")

# Unzip Bus Trips
os.system("unzip -o data/bus/gtfs.zip trips.txt -d data/bus")

--2024-10-05 18:19:44--  https://gitlab.com/LACMTA/gtfs_rail/-/raw/master/stops.txt
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35215 (34K) [text/plain]
Saving to: ‘data/rail/stops.txt’

     0K .......... .......... .......... ....                 100% 8.88M=0.004s

2024-10-05 18:19:44 (8.88 MB/s) - ‘data/rail/stops.txt’ saved [35215/35215]

--2024-10-05 18:19:44--  https://gitlab.com/LACMTA/gtfs_rail/-/raw/master/trips.txt
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 412031 (402K) [text/plain]
Saving to: ‘data/rail/trips.txt’

     0K .......... .......... .......... .......... .......... 12% 4.72M 0s
    50K .......... .......... .......... .......... 

Archive:  data/bus/gtfs.zip
  inflating: data/bus/stops.txt      
Archive:  data/bus/gtfs.zip
  inflating: data/bus/trips.txt      


... 97% 6.86M 0s
 21100K .......... .......... .......... .......... .......... 97% 5.05M 0s
 21150K .......... .......... .......... .......... .......... 98% 6.71M 0s
 21200K .......... .......... .......... .......... .......... 98% 6.09M 0s
 21250K .......... .......... .......... .......... .......... 98% 5.03M 0s
 21300K .......... .......... .......... .......... .......... 98% 1.90M 0s
 21350K .......... .......... .......... .......... .......... 99% 7.21M 0s
 21400K .......... .......... .......... .......... .......... 99% 13.6M 0s
 21450K .......... .......... .......... .......... .......... 99%  223M 0s
 21500K .......... .......... .......... .......... .......... 99% 4.08M 0s
 21550K .......... .......... .......... .......... .......... 99% 23.7M 0s
 21600K ......                                                100% 66.4M=3.8s

2024-10-05 18:19:48 (5.56 MB/s) - ‘data/bus/gtfs.zip’ saved [22124906/22124906]



0

## Trips

Add `wheelchair_accessible` column in trips.txt.

Valid options for `wheelchair_accessible`:

```
0 or empty - No accessibility information for the trip.
1 - Vehicle being used on this particular trip can accommodate at least one rider in a wheelchair.
2 - No riders in wheelchairs can be accommodated on this trip.
```

In [18]:
# Create dataframes for trips.txt files
bus_trips_df = pd.read_csv("data/bus/trips.txt")
rail_trips_df = pd.read_csv("data/rail/trips.txt")

In [19]:
# Add a column named "wheelchair_accessible" and set to 1
bus_trips_df["wheelchair_accessible"] = 1
rail_trips_df["wheelchair_accessible"] = 1

In [20]:
# Save the updated dataframes
bus_trips_df.to_csv("output/bus/trips.txt", index=False)
rail_trips_df.to_csv("output/rail/trips.txt", index=False)

## Stops

Add `wheelchair_boarding` column in stops.txt.

Valid options for `wheelchair_boarding`:

> For parentless stops:
>
> * 0 or empty - No accessibility information for the stop.
> * 1 - Some vehicles at this stop can be boarded by a rider in a wheelchair.
> * 2 - Wheelchair boarding is not possible at this stop.
>
> For child stops:
>
> * 0 or empty - Stop will inherit its wheelchair_boarding behavior from the parent station, if specified in the parent.
> * 1 - There exists some accessible path from outside the station to the specific stop/platform.
> * 2 - There exists no accessible path from outside the station to the specific stop/platform.
>
> For station entrances/exits:
>
> * 0 or empty - Station entrance will inherit its wheelchair_boarding behavior from the parent station, if specified for the parent.
> * 1 - Station entrance is wheelchair accessible.
> * 2 - No accessible path from station entrance to stops/platforms.
> 

Start by initializing values to `0` to indicate by default that there is no accessibility information is available for the stop.

Data will then be added on a case by case basis.

In [21]:
# Create dataframes for stops.txt flies
rail_stops_df = pd.read_csv("data/rail/stops.txt")
bus_stops_df = pd.read_csv("data/bus/stops.txt")

In [22]:
# Add wheelchair_boarding column and set default value to 0 to indicate no accessibility information is available
rail_stops_df["wheelchair_boarding"] = 0
bus_stops_df["wheelchair_boarding"] = 0

### Rail

Start with A Line Stations


In [25]:
# find rows where stop_name contains Irwindale
irwindale_rail_stops_df = rail_stops_df[rail_stops_df["stop_name"].str.contains("Irwindale", case=False)]

irwindale_rail_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,tpis_name,wheelchair_boarding
378,80425,80425,Irwindale Station,NaN,34.129048,-117.932506,NaN,0,80425S,Irwndale,0
379,80425S,80425S,Irwindale Station,NaN,34.129048,-117.932506,NaN,1,NaN,NaN,0
380,80425A,80425A,Irwindale Station - West Ramp,NaN,34.128770,-117.933480,NaN,2,80425S,NaN,0
381,80425B,80425B,Irwindale Station - West Stairs,NaN,34.128840,-117.933290,NaN,2,80425S,NaN,0


In [23]:
# Save the updated dataframes
rail_stops_df.to_csv("output/rail/stops.txt", index=False)
bus_stops_df.to_csv("output/bus/stops.txt", index=False)